In [ ]:
project_path = '/home/jupyter'
import os
import sys
sys.path.append(project_path)
sys.path.append(f'{project_path}/ft_events/src/')
import importlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from google.cloud import bigquery
from google.cloud import storage
import geopandas as gpd

from fintrans_toolbox.src import table_utils as t

from make_data import load_data

In [ ]:
def add_game_periods(date_list):
    fillcolor = "red"
    
    for i in date_list:
        start_date = i[0:8] + "01"
        end_date = i[0:8] + "02"


        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=1,
            layer="below",
            line_width=0,
        )

# Pick team, event place and date

In [ ]:
#2022 - Wigan, Huddersfield Challenged cup final
# arena = 'N'
# team_1 = 'WN'
# team_2 = 'HD'
# teams = [team_1, team_2]
# event_date = '2022-05-28'

# Feb 2024 - Chelsea v Liverpool @ Wembley
arena = 'HA'
team_1 = 'L'
team_2 = 'SW'
teams = [team_1, team_2]
event_date = '2024-02-01'

# Read in and manipulate data

In [ ]:
rphst = load_data.load_rphst(cardholder_level = 'POSTAL_AREA', merchant_level = 'POSTAL_AREA', month_quarter = 'Month')
rphst = t.create_date_time(rphst)
rphst = rphst[rphst['mcg'] =='All']

In [ ]:
all_rphst = load_data.load_rphst(cardholder_level = 'All', merchant_level = 'POSTAL_AREA', month_quarter = 'Month')
all_rphst = t.create_date_time(all_rphst)
all_rphst = all_rphst[all_rphst['mcg'] =='All']
all_arena = all_rphst[all_rphst['merchant_location'] == arena].reset_index(drop = True)

In [ ]:
playing = rphst[rphst['cardholder_location'].isin(teams)].reset_index(drop = True)
playing = playing[playing['merchant_location'] == arena].reset_index(drop = True)

# index data

In [ ]:
def index(df):
    df = df[df['mcg'] == 'All']
    df = df.groupby(['date_time','cardholder_location'])['spend'].sum().reset_index()
    df = df.sort_values(['date_time']).reset_index(drop = True)
    df['index_spend'] = df.groupby(['cardholder_location'])['spend'].transform(lambda x: x / (x.iloc[0:11].mean(axis = 0))*100)

    return(df)

In [ ]:
index_playing = index(playing)
index_arena = index(all_arena)
index_df = pd.concat([index_playing,index_arena])

# plot charts

In [ ]:
def plot(df, y_value, remove_all):
    if remove_all == True:
        df = df[df['cardholder_location'] != 'All']
    else:
        pass
    
    fig = px.line(
    df,
    x="date_time",
    y=y_value,
    color="cardholder_location",
    title="",
    template = 'simple_white'
    )

# add date colour
    fillcolor = "red"
    start_date = event_date[0:8] + "01"
    end_date = event_date[0:8] + "02"
    

    fig.add_shape(
        type="rect",
        xref="x",
        yref="paper",
        x0=start_date,
        y0=0,
        x1=end_date,
        y1=1,
        fillcolor=fillcolor,
        opacity=1,
        layer="below",
        line_width=0,
    )
    
    fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
    
    fig.show()

In [ ]:
plot(df = index_df, y_value = 'spend', remove_all = True)

---------------------------

# Major UK stadium overviews

### Select stadium postal area:

Wembley Stadium - <b>HA9

Old Trafford - <b>M16

Tottenham - <b>N17

London West Ham - <b>E20

Anfield - <b>L4

Emirates (Arsenal) - <b>N7

Etihad - <b>M11

St James' Park - <b>NE1

In [ ]:
stadium_area = 'N'
mcg = 'All'

In [ ]:
away_team = 'L'

In [ ]:
rphst = load_data.load_rphst(cardholder_level = 'POSTAL_AREA', merchant_level = 'POSTAL_AREA', month_quarter = 'Month')
rphst = t.create_date_time(rphst)
rphst = rphst[rphst['mcg'] ==mcg]
all_stadium = rphst[rphst['merchant_location'] == stadium_area].reset_index(drop = True)

In [ ]:
fig = px.line(
all_stadium[(all_stadium['cardholder_location'] == away_team)],

x="date_time",
y='spend',
color = 'cardholder_location',
title=f"Sum spend of {away_team} cardholders at {stadium_area}",
height = 500,
template = 'simple_white',

)

fig.update_layout(legend_title_text='Postal area:', title_subtitle_text=f'MCG = {mcg}')
# add_game_periods(date_list = ['2019-11-01', 
#                               '2022-02-01', 
#                               '2023-04-01',
#                              '2023-10-01'] )

fig.show()

---------------------------

# Co-op live

In [ ]:
co_op = load_data.load_rphst(cardholder_level = 'All', merchant_level = 'POSTAL_DISTRICT', month_quarter = 'Month')
co_op = t.create_date_time(co_op)

co_op = co_op[co_op['merchant_location'] == 'M11'].reset_index(drop = True)

In [ ]:
fig = px.line(
co_op,

x="date_time",
y='spend',
color = 'mcg',
title=f"Sum spend at M11",
height = 500,
template = 'simple_white',

)

fig.show()

---------------------------

In [ ]:
district_choice = 'LU1'

In [ ]:
df = load_data.load_rphst(cardholder_level = 'All', merchant_level = 'POSTAL_DISTRICT', month_quarter = 'Month')
df = t.create_date_time(df)

df = df[df['merchant_location'] == district_choice].reset_index(drop = True)

In [ ]:
fig = px.line(
df,

x="date_time",
y='spend',
color = 'mcg',
title=f"Sum spend {district_choice}",
height = 500,
template = 'simple_white',

)

fig.show()